In [1]:
import pandas as pd 
import numpy as np
from WindPy import w
from tqdm.notebook import tqdm

In [2]:
# 提取全部仓位数据, 主要针对半年报
def stockproportion(date= '20210331', code = '003567'):
    w.start()
    data = w.wset("allfundhelddetail",f"rptdate={date};windcode={code}.OF")
    stocks = pd.DataFrame(data.Data).T
    stocks.columns = data.Fields
    stocks = stocks[['stock_code', 'stock_name', 'proportiontototalstockinvestments']]
    stocks.columns = ['code', 'name', 'weight']
    w.close()
    return stocks

# 提取前十大重仓股数据， 针对季报
def stockproportion2(fundcodes, date='20210331', numstock=10):
#     w.start()
    if not isinstance(fundcodes, list):
        fundcodes = list(fundcodes)
    fundcodes = [x if x.endswith('OF') else x+'.OF' for x in fundcodes]
    if len(fundcodes) > 1:
        codes = ','.join(fundcodes)
    else:
        codes = fundcodes
    datatemp = pd.DataFrame([], index=fundcodes)
    for x in range(1, numstock + 1):
        data = w.wss(codes, "prt_topstockcode,prt_topstockvalue", f"rptDate={date};order={x};unit=1")
        fundtemp = pd.DataFrame(data.Data).T
        fundtemp.columns = [f'第{x}权重股代码', f'第{x}权重股市值']
        fundtemp.index = data.Codes
        datatemp = pd.concat([datatemp, fundtemp], axis=1)
#     w.close()
    datatemp.to_excel('results/基金重仓股.xlsx')
    stockcode = datatemp[[f'第{x}权重股代码' for x in range(1, numstock + 1)]].T
    stockcode.reset_index(drop= True, inplace=True)
    stockcode.columns = [x+'code' for x in fundcodes]

    stockvalue = datatemp[[f'第{x}权重股市值' for x in range(1, numstock + 1)]].T
    stockvalue.reset_index(drop= True, inplace=True)
    stockvalue.fillna(0, inplace=True)

    stockweight = stockvalue.apply(lambda x: x/sum(x), axis=0)
    stockweight.columns = [x+'weight' for x in fundcodes]
    datatemp2 = pd.concat([stockcode, stockweight], axis=1)
    result = dict()
    for x in fundcodes:
        table = datatemp2[[x+'code', x+'weight']]
        table.columns = ['code', 'weight']
        table.dropna(how='any', inplace=True)
        result[x] = table
    return result


In [3]:
# 汇总基金持仓股票的行业权重和涨跌幅
def fundinfo(fundtable, stocktable):
    fundstock = fundtable.merge(stocktable, how='left', left_on='code', right_on='证券代码')
    fundstock = fundstock[['code', '证券简称','weight','所属申万行业名称','区间涨跌幅']]
    fundstock.columns = ['code', 'name','weight','swindustry','return3m']
    fundstock['weight'] = fundstock['weight']
    fundstock['wr'] = fundstock['weight'] * fundstock['return3m']
    fundindus = fundstock.groupby(by='swindustry')['weight','wr'].sum()
    fundindus[f'return'] = fundindus[f'wr']/fundindus['weight']
    return fundindus

In [4]:
# 运行Brinson模型
def brinson(indus, fund):
    mergetable = indus.merge(fund, how='left', left_on='所属申万行业名称',right_index=True)
    mergetable.fillna(0, inplace=True)
    mergetable['AR'] = (mergetable['weight']- mergetable['权重'])* mergetable['3个月涨跌幅']
    mergetable['SR'] = (mergetable['return']- mergetable['3个月涨跌幅'])* mergetable['weight']
    result = mergetable[['AR','SR']].apply(lambda x: sum(x), axis=0)
#     print(mergetable)
    return result
    

In [5]:
dates = []
for x in range(2018,2022):
    for y in ['0331', '0630', '0930', '1231']:
        dates.append(str(x)+y)
dates = dates[1:-3]
print(dates)

['20180630', '20180930', '20181231', '20190331', '20190630', '20190930', '20191231', '20200331', '20200630', '20200930', '20201231', '20210331']


In [6]:

def brinsonall(codes, dates=dates):
    w.start()
    arset = pd.DataFrame()
    srset = pd.DataFrame()
    codes = [x if x.endswith('OF') else x+'.OF' for x in codes]
#     print(codes)
    for date in tqdm(dates):
        funds = stockproportion2(codes,date=date)
        allstock = pd.read_excel(f'data/A股{date}.xlsx')
        allstock.dropna(subset=['所属申万行业名称'], inplace=True)
        allstock['证券代码'] = allstock['证券代码'].map(lambda x: x[:-3])
        indus = pd.read_excel(f'data/基准fund{date}.xlsx')[['所属申万行业名称', '权重','3个月涨跌幅']]
        for key in tqdm(funds):
            fundindus = fundinfo(fundtable=funds[key], stocktable=allstock)
            ARSR = brinson(indus, fundindus)
            arset.loc[key, date] = ARSR['AR']
            srset.loc[key, date] = ARSR['SR']
    w.close()
    return {'arset':arset, 'srset':srset}

In [7]:
f = pd.read_excel('data/偏股基金.xlsx')
codes = f['证券代码']
result = brinsonall(codes)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
['001643.OF', '003853.OF', '000828.OF', '000336.OF', '001679.OF', '005968.OF', '002190.OF', '003984.OF', '000209.OF', '002083.OF', '165516.OF', '003834.OF', '001704.OF', '161903.OF', '001856.OF', '001245.OF', '002168.OF', '001616.OF', '004812.OF', '001606.OF', '002669.OF', '001410.OF', '001216.OF', '002780.OF', '004997.OF', '001951.OF', '001532.OF', '000263.OF', '005689.OF', '001156.OF', '003516.OF', '005136.OF', '005927.OF', '000409.OF', '005805.OF', '519778.OF', '001054.OF', '005630.OF', '519133.OF', '004640.OF', '003230.OF', '481010.OF', '002939.OF', '360016.OF', '001811.OF', '003961.OF', '400015.OF', '002846.OF', '519002.OF', '000601.OF', '000541.OF', '540008.OF', '001104.OF', '005668.OF', '000924.OF', '001224.OF', '001126.OF', '290011.OF', '673060.OF',

  0%|          | 0/12 [00:00<?, ?it/s]

c:\users\严书航\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  0%|          | 0/987 [00:00<?, ?it/s]

c:\users\严书航\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


...

  0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/987 [00:00<?, ?it/s]

In [8]:
writer = pd.ExcelWriter('results/ARSR2.xlsx')
for x in ['arset', 'srset']:
    table = result[x]
    a = table.apply(lambda x: len(x[x>0]) / len(x), axis=1)
    b = table.apply(lambda x: np.mean(x), axis=1)
    c = table.apply(lambda x: np.median(x), axis=1)
    table['winrate'] = a
    table['average'] = b
    table['median'] =c
    table = f.merge(table, how='right', left_on='证券代码', right_index=True)
    table.to_excel(writer, sheet_name=x)
writer.close()